In [1]:
import os

In [2]:
%pwd

'e:\\Data_Science_Project\\TalentGuard\\research'

In [3]:
os.chdir("../")

In [6]:
%pwd

'e:\\Data_Science_Project\\TalentGuard'

In [28]:
from dataclasses import dataclass
from pathlib import Path
from typing import Optional


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators : int
    max_depth : Optional[int]
    min_samples_split : int
    class_weight : str
    target_column : int

In [29]:
from TalentGuard.constants import *
from TalentGuard.utils.common import read_yaml, create_directories

In [30]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators =  params.n_estimators,
            max_depth =  params.max_depth,
            min_samples_split  =  params.min_samples_split,
            class_weight =  params.class_weight,
            target_column = schema.name
            
        )
        return model_trainer_config

In [31]:
import pandas as pd
import os
from TalentGuard import logger
from sklearn.ensemble import RandomForestClassifier
import joblib

In [32]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]


        lr = RandomForestClassifier(n_estimators=self.config.n_estimators,
                                    max_depth=self.config.max_depth,
                                    min_samples_split=self.config.min_samples_split,
                                    class_weight=self.config.class_weight)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

In [33]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-11 14:20:18,609: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-11 14:20:18,614: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-11 14:20:18,623: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-11 14:20:18,627: INFO: common: created directory at: artifacts]
[2025-01-11 14:20:18,630: INFO: common: created directory at: artifacts/model_trainer]


e:\Data_Science_Project\TalentGuard\env\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
